# **Preprocesamiento, limpieza, normalización y creación de ventanas para modelos secuenciales (CNN, LSTM, CNN-LSTM)**

In [ ]:
# ============================================================
# Importación de librerías y configuración global
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from sklearn.preprocessing import StandardScaler

SEED = 42
np.random.seed(SEED)

print("Librerías cargadas y semilla fijada.")


Librerías cargadas y semilla fijada.


Definimos las rutas usando Path, que funciona igual en Windows/Colab/Linux. No hay rutas absolutas que rompan el notebook. Luego leemos los tres archivos igual que en el Notebook 01, manteniendo consistencia.

In [ ]:
# ============================================================
# CARGA AUTOMÁTICA DE DATOS DESDE GOOGLE DRIVE (SIN MONTAR)
# ============================================================

import pandas as pd

url_etios = "https://drive.google.com/uc?export=download&id=1FExXO0NzOWPCGeLVerHClndAMaASocSc"
url_figo  = "https://drive.google.com/uc?export=download&id=10SvGG6Rj0xHwwO4bkutR3DY_wipxtBN7"
url_rrv   = "https://drive.google.com/uc?export=download&id=1U-SjwjE3AfPpMGrhWVq-ecsQwNjVseO-"

etios = pd.read_csv(url_etios)
figo  = pd.read_csv(url_figo)
rrv   = pd.read_csv(url_rrv)

print("Datasets cargados correctamente:")
print("Etios:", etios.shape)
print("Figo: ", figo.shape)
print("RRV:  ", rrv.shape)

print("Datasets cargados correctamente.")

Datasets cargados correctamente:
Etios: (272008, 47)
Figo:  (167559, 47)
RRV:   (0, 3)
Datasets cargados correctamente.


### Unificación en un solo dataset

Esto es normal en proyectos de series temporales cuando todos representan el mismo tipo de sistema físico. Se crea un solo dataframe para simplificar escalado y ventaneo. ignore_index=True evita conflictos de índices.

In [ ]:
# ============================================================
# Unificación de datasets
# ============================================================

df = pd.concat([etios, figo, rrv], ignore_index=True)

print("Dataset unificado con forma:", df.shape)


Dataset unificado con forma: (439567, 50)


**Identificar variable objetivo NOx**: busca automáticamente la columna con NOx, independientemente de cómo se llame (NOX, nox_ppm, etc.).

In [ ]:
# ============================================================
# Detección automática del nombre de la columna objetivo (NOx)
# ============================================================

target_col = None
for col in df.columns:
    if "nox" in col.lower():
        target_col = col
        break

print("Variable objetivo detectada:", target_col)


Variable objetivo detectada: NOx_wet_conc


**Selección de variables numéricas**: Escogemos todas las columnas numéricas como features iniciales. CNN y LSTM solo trabajan con datos numéricos.

In [ ]:
# ============================================================
# Selección de variables numéricas para el modelo
# ============================================================

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Asegurarnos de que la variable objetivo está incluida
assert target_col in numeric_cols, "NOx no es numérica, revisar CSV."

print("Variables numéricas total:", len(numeric_cols))
print(numeric_cols)


Variables numéricas total: 45
['trip', 'driver', 'route', 'load', 'gps_lat', 'gps_lon', 'gps_alt', 'gps_speed', 'humidity', 'pressure', 'temp', 'rpm', 'speed_vehicle', 'throttle', 'manifold_pressure', 'manifold_temp', 'coolant_temp', 'fuel_flow', 'fuel_rate', 'air_fuel_ratio', 'exh_humidity', 'exh_mass_flow', 'exh_flow_scfm', 'exh_flow_ls', 'exh_temp', 'CO2_amb_conc', 'CO_amb_conc', 'NO_amb_conc', 'NO2_amb_conc', 'O2_amb_conc', 'CO2_wet_conc', 'CO_wet_conc', 'NO_wet_conc', 'NO2_wet_conc', 'NOx_wet_conc', 'O2_wet_conc', 'CO2_mass', 'CO_mass', 'NO_mass', 'NO2_mass', 'NOx_mass', 'O2_mass', 'NO_mass_cor', 'NO2_mass_cor', 'NOx_mass_cor']


**Tratamiento básico de NaNs**: Rellenamos valores faltantes propagando hacia adelante y atrás. Es un método muy común en series temporales.

In [ ]:
# ============================================================
# Limpieza inicial: manejo simple de NaN por forward-fill
# ============================================================

df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')

print("NaNs tratados.")


/tmp/ipython-input-3311405938.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')


NaNs tratados.


**Separación: features (X) y objetivo (y)**: Creamos matrices independientes:

X: todas las variables numéricas

y: solo NOx

In [ ]:
# ============================================================
# Separación de X e y
# ============================================================

X = df[numeric_cols].copy()
y = df[target_col].copy()

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (439567, 45)
y shape: (439567,)


**Split por trayectos (sin fuga temporal)**: para evitar que el modelo vea el futuro se hace un split temporal puro: el pasado → entrenamiento; el futuro → validación/test. Crucial para evitar “data leakage”.

In [ ]:
# ============================================================
# Split secuencial train/val/test (sin mezclado)
# ============================================================

n = len(df)
train_end = int(0.6 * n)
val_end   = int(0.8 * n)

X_train = X.iloc[:train_end].values
y_train = y.iloc[:train_end].values

X_val   = X.iloc[train_end:val_end].values
y_val   = y.iloc[train_end:val_end].values

X_test  = X.iloc[val_end:].values
y_test  = y.iloc[val_end:].values

print("Train:", X_train.shape)
print("Val:  ", X_val.shape)
print("Test: ", X_test.shape)


Train: (263740, 45)
Val:   (87913, 45)
Test:  (87914, 45)


**Normalización (solo se ajusta con train)**: se ajusta el escalador únicamente con el set de entrenamiento.
Luego se aplica a val/test → buenas prácticas de ML.

In [ ]:
# ============================================================
# Normalización estándar: fit solo en train
# ============================================================

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("Normalización completada.")


Normalización completada.


**Función para crear ventanas temporales**: Convierte una serie en muestras del tipo:

[ t, t+1, ..., t+T ] → y en t+T

In [ ]:
# ============================================================
# Función de ventaneo (crea tensores para modelos secuenciales)
# ============================================================

def create_windows(X, y, window_size):
    X_w, y_w = [], []
    for i in range(len(X) - window_size):
        X_w.append(X[i:i+window_size])
        y_w.append(y[i+window_size])
    return np.array(X_w), np.array(y_w)

print("Función create_windows definida.")


Función create_windows definida.


float64 → float32 reduce memoria al 50%, acelera entrenamiento y evita errores.

In [ ]:
# ============================================================
# Conversión a float32 para reducir memoria a la mitad
# ============================================================

X_train_scaled = X_train_scaled.astype(np.float32)
X_val_scaled   = X_val_scaled.astype(np.float32)
X_test_scaled  = X_test_scaled.astype(np.float32)

y_train = y_train.astype(np.float32)
y_val   = y_val.astype(np.float32)
y_test  = y_test.astype(np.float32)

print("Datos convertidos a float32.")

Datos convertidos a float32.


Esta función:

- No genera un arreglo gigante

- Usa ventanas en streaming.

- Permite batch automático.

- Sirve para todos tus modelos secuenciales (CNN, LSTM, CNN-LSTM).

In [ ]:
# ============================================================
# Creación correcta de ventanas en tf.data para pares (X, y)
# ============================================================

import tensorflow as tf

def make_window_dataset(X, y, window, batch, shuffle=True):
    """
    Construye un dataset de ventanas seguras para TensorFlow:
    Entrada:
        X: matriz numpy (n_samples, n_features)
        y: vector numpy (n_samples,)
    Salida:
        Dataset con batches de shape:
            X_batch: (batch, window, n_features)
            y_batch: (batch,)
    """

    # Datasets independientes
    X_ds = tf.data.Dataset.from_tensor_slices(X)
    y_ds = tf.data.Dataset.from_tensor_slices(y)

    # Ventanas separadas para X e y
    X_w = X_ds.window(window, shift=1, drop_remainder=True)
    y_w = y_ds.window(window, shift=1, drop_remainder=True)

    # Convertimos cada ventana en un batch real
    X_w = X_w.flat_map(lambda w: w.batch(window))
    y_w = y_w.flat_map(lambda w: w.batch(window))

    # Emparejar ventanas X_window, y_window
    ds = tf.data.Dataset.zip((X_w, y_w))

    # Para cada ventana, dejamos:
    #   X_window --> (window, n_features)
    #   y_window --> tomamos el último valor como etiqueta
    ds = ds.map(lambda x, y: (x, y[-1]))

    # Opcional: shuffle solo en train
    if shuffle:
        ds = ds.shuffle(5000, seed=SEED)

    # Batch final + prefetch
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)

    return ds



**Crear datasets listos para entrenamiento**

In [ ]:
# ============================================================
# Creación de datasets para train, val y test
# ============================================================

WINDOW = 30
BATCH  = 64

train_ds = make_window_dataset(X_train_scaled, y_train, window=WINDOW, batch=BATCH, shuffle=True)
val_ds   = make_window_dataset(X_val_scaled,   y_val,   window=WINDOW, batch=BATCH, shuffle=False)
test_ds  = make_window_dataset(X_test_scaled,  y_test,  window=WINDOW, batch=BATCH, shuffle=False)

print("Datasets creados correctamente.")

Datasets creados correctamente.


**Revisión rápida del shape (para confirmar)**

In [ ]:
for Xb, yb in train_ds.take(1):
    print("Batch X:", Xb.shape)  # (batch, window, n_features)
    print("Batch y:", yb.shape)  # (batch,)

Batch X: (64, 30, 45)
Batch y: (64,)


**Guardar metadatos del preprocesamiento**:
Guardamos solo lo necesario para reconstruir el pipeline en Colab o en inferencia:

- Escalador

- Columnas

- Tamaño de ventana

- Sin ocupar RAM innecesaria.

In [ ]:
# ============================================================
# Guardar metadatos del preprocesamiento
# ============================================================

preprocessing_info = {
    "feature_names": numeric_cols,
    "target": target_col,
    "window": WINDOW,
    "scaler_mean": scaler.mean_.tolist(),
    "scaler_scale": scaler.scale_.tolist(),
}

import json
with open("preprocessing_info.json", "w") as f:
    json.dump(preprocessing_info, f, indent=4)

print("preprocessing_info.json guardado correctamente.")


preprocessing_info.json guardado correctamente.


**Guardar splits (solo escalados)**:

Aunque usamos tf.data para ventaneo, es útil guardar los splits escalados para reproducibilidad o para crear ventanas distintas (ej. T=60 luego).

In [ ]:
# ============================================================
# Guardar los splits escalados para futuros notebooks
# ============================================================

np.savez("splits_escalados.npz",
         X_train=X_train_scaled,
         y_train=y_train,
         X_val=X_val_scaled,
         y_val=y_val,
         X_test=X_test_scaled,
         y_test=y_test)

print("splits_escalados.npz guardado correctamente.")


splits_escalados.npz guardado correctamente.


**Inspección rápida del dataset final**

Para asegurarnos de que todo funciona, añadimos una celda que imprime shapes y ejemplos.

In [ ]:
# ============================================================
# Validación final del preprocesamiento
# ============================================================

print("Shapes originales:")
print("  X_train:", X_train_scaled.shape)
print("  X_val:  ", X_val_scaled.shape)
print("  X_test: ", X_test_scaled.shape)
print()

# Probar batch real
for Xb, yb in train_ds.take(1):
    print("Ejemplo batch X:", Xb.shape)
    print("Ejemplo batch y:", yb.shape)
    break

print("\nTodo listo para usar en modelos CNN, LSTM y CNN-LSTM.")


Shapes originales:
  X_train: (263740, 45)
  X_val:   (87913, 45)
  X_test:  (87914, 45)

Ejemplo batch X: (64, 30, 45)
Ejemplo batch y: (64,)

Todo listo para usar en modelos CNN, LSTM y CNN-LSTM.



# **Conclusión del notebook**


En este notebook se completó el preprocesamiento integral del conjunto de datos, garantizando un pipeline reproducible y compatible con modelos secuenciales de Deep Learning. Se unificaron y limpiaron los registros, se manejaron adecuadamente valores faltantes, y se aplicó una normalización estricta sin fuga de información. Los datos se dividieron temporalmente en conjuntos de entrenamiento, validación y prueba, y se implementó un sistema de ventaneo eficiente mediante tf.data, capaz de generar secuencias para CNN, LSTM y arquitecturas híbridas sin comprometer la memoria. Finalmente, se guardaron los metadatos y splits necesarios para asegurar continuidad en los siguientes notebooks. Con esto, el dataset queda completamente preparado para iniciar el desarrollo y evaluación de los modelos de Deep Learning.

